# Data Mining

Data Mining adalah teknik untuk mengeksplorasi, mengakuisisi, dan memeriksa data. Berdasarkan jenis data, metode yang digunakan bisa beraneka ragam. Pada tutorial ini dikenalkan teknik akuisisi data dari Kaggle, website, dan API.

## 1. Kaggle Datasets

Kaggle adalah komunitas online untuk data scientist dan saat ini dimiliki oleh Google (sejak 2017).
In short, Kaggle is a sort of online community for data scientists, now owned by Google (as of 2017).Komunitas ini memungkinkan para penggunanya (disebut Kagglers) untuk menerbitkan set data, membangun model dalam lingkungan ilmu data berbasis web, mengikuti kompetisi tantangan ilmu data, bertukar ide/kode, dll. Sekitar waktu Google mengakuisisi Kaggle, basis penggunanya melampaui 1.000.000 pengguna terdaftar, yang mencakup 190+ negara, membentuk komunitas ilmu data terbesar dan paling beragam di dunia.

Khususnya bagi para mahasiswa data science, ini adalah sumber materi yang sangat berguna, bukan hanya karena banyaknya set data dunia nyata dan masalah ilmu data, tetapi juga karena kode sumber, ide, dll. yang dibagikan oleh Kagglers lainnya. Di bagian ini, saya akan menunjukkan salah satu dari (banyak) cara untuk menggunakan Kaggle untuk proyek Anda sendiri.


### 1.1. Getting Started

#### Sign up
Buat akun Kaggle https://www.kaggle.com/

#### Create an API Token
Buat API Token, akses `My Account`, menu `Settings`.
![User settings menu](figures/kaggle_account.png)

Cari tombol '`Create New API Token`'.
![Create New Kaggle API Token](figures/create_new_kaggle_api_token.png)

Maka akan dibuatkan file `kaggle.json` dan otomatis terunduh. Isi dari file tersebut adalah
```json
{"username":"bigdata","key":"0123456789abcdefghijklmn"}
```

### 1.2. Install Python Kaggle Library

Sekarang instal modul `kaggle` di jupyter notebook lokal atau fi Google colab.

In [1]:
# Install kaggle
!pip install kaggle

Salin isi dari file `kaggle.json`, lalu spesifikasikan ke variable, misalkan `token`

In [2]:
# type your Kaggle API token here
token = {"username":"kristinwulandari","key":"6c724e0ac25f9b0118dd26c9daa83296"}

Lalu jalankan kode berikut ini. Kode ini pada dasarnya untuk membuat file `kaggle.json`, dan isi dari file tersebut adalah Token API. Lalu, file tersebut diset aksesibilitasnya.

In [3]:
# This cell is only for advanced users. Run this cell and you can safely move on to the next cell
import os
from pathlib import Path
import json
import platform

# creates and places the token file at a desired location
home = str(Path.home())
kaggle_root = os.path.join(*[home, '.kaggle'])
os.mkdir(kaggle_root)
with open(os.path.join(*[kaggle_root, 'kaggle.json']), 'w') as file:
    json.dump(token, file)

# make the key file accessible only to the owner
if platform.system() == 'Windows':
    !attrib -R {os.path.join(*[kaggle_root, 'kaggle.json'])}
else:
    !chmod 600 {os.path.join(*[kaggle_root, 'kaggle.json'])}

### Alternative Method (Colab Only)

Alternatif lainnya, jika tidak ingin menggunakan kode program di atas, di Google colab bisa menggunakan file upload berikut ini. pload file kabble.json

In [4]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


Lalu jalan kode berikut.

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


### 1.3. Downloading a Data Set from Kaggle

Unduh dataset dari kaggle menggunakan kode sederhana berikut.
```bash
!kaggle datasets download -d <path-to-dataset> -p <download-location>
```

`path-to-dataset` mengikuti alamat `http://www.kaggle.com/`. Misalkan untuk data `avocado prices` di kaggle yang berada di `https://www.kaggle.com/neuromusic/avocado-prices` maka path yang digunakan adalah `neuromusic/avocado-prices`.

In [6]:
!kaggle datasets download -d neuromusic/avocado-prices -p data

Dataset URL: https://www.kaggle.com/datasets/neuromusic/avocado-prices
License(s): ODbL-1.0
  0% 0.00/629k [00:00<?, ?B/s]
100% 629k/629k [00:00<00:00, 76.9MB/s]


Sebagian besar dataset dikompresi misalnya dengan format zip atau tar.gz. Pengguna Mac/Linux cukup jalankan perintah ini:
```bash
!unzip ./data/avocado-prices.zip -d ./data
```
untuk extract file. Kalau anda pengguna Windows, selamat, prosesnya lebih ribet, lihat di [sini](https://stackoverflow.com/questions/1021557/how-to-unzip-a-file-using-the-command-line).

In [7]:
# Unzipping files.
if platform.system() == 'Windows':
    this_file_path = !echo %cd%
    data_path = os.path.join(*[this_file_path[0], 'data'])
    print('[IMPORTANT] No automatic unzipping supported on Windows.')
    print('You have to open `File Explorer` and manually unzip `' + data_path + '\\avocado-prices.zip`')
    print('Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:')
    print('|- ica03')
    print('    |- data')
    print('        |- avocado.csv')
    print('    |- Data_Mining.ipynb')
else:
    !unzip ./data/avocado-prices.zip -d ./data

Archive:  ./data/avocado-prices.zip
  inflating: ./data/avocado.csv      


## 2. Web Scraping with BeautifulSoup

Kadang kala analisa data dilakukan pada data spesifik. Jika demikian perlu menulis aplikasi Crawler sendiri. Di sini sigunakan modul `BeautifulSoup`.

Berikut contohnya mengekstraksi data penerbangan di (https://flycid.com/flight-status/). Buka file link tersebut untuk melihat isinya.

![Cedar Rapids Airport Webpage](figures/cid.png)

### 2.1. Anatomy of a Web Page

Setiap orang memiliki pendekatan sendiri untuk eksplorasi data. Pengguna Chrome atau Firefox lakukan inspeksi struktur web dengan menekan `ctrl (cmd) + shift + I` atau `F12`. Jika menggunakan Safari gunakan Web Inspektor menggunakan `cmd + shift + I`. Untuk browser lainnya cari informasinya di internet.

Inspesi di Chrome menampilkan tampilan berikut:

![Developer Tools in Chrome](figures/dev_tools.png)

Terlihat skrip halam web menggunakan HTML. Jika cursor berada di atas skrip maka bagian di web akan dihighlight, sebagaimana berikut:

![Developer Tools can Highlight Areas](figures/code_highlight.png)

Sekarang bagian yang agak membosankan, tapi perlu. Cari bagian iframe, di situ ada link yang menjadi sumber informasi halaman web.

![Actual Flight Information in iframe](figures/frame.png)

Berdasarkan gambar di atas, sumber informasi adalah (https://aerocloud-us-fids-widgets.s3.us-east-2.amazonaws.com/cid.html) within the airport web page as if it is a part of the web page. Kopi, lalu akses di kode program menggunakan modul `requests`.

### 2.2. Get and Parse HTML

Sekarang akses link sumber informasinya:

In [8]:
import requests
page = requests.get("https://aerocloud-us-fids-widgets.s3.us-east-2.amazonaws.com/cid.html")

Parsing menggunakan BeautifulSoup

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [10]:
print(soup.prettify())

<head>
 <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
 <script>
  let cityFilter = "";
    let airlineFilter = "";
    let tabIndex = 0;

    const setCityFilter = (city, name) => {
      
      cityFilter = city;
      document.getElementById("cityFilter").innerHTML = name
        ? "[" + name + "]"
        : "";
      document.getElementById("city").style.display = city ? "" : "none";
      if (city && document.body.scrollTop > 100) {
        document.body.scrollTop = 100;
      }
    };
    const setAirlineFilter = (airline) => {
      
      airlineFilter = airline;
      const airlineName =
        airline === "AA"
          ? "American Airlines"
          : airline === "DL"
          ? "Delta Air Lines"
          : airline === "UA"
          ? "United Airlines"
          : airline === "F9"
          ? "Frontier Airlines"
          : airline === "NK"
          ? "Spirit Airlines"
          : airline === "4B"
          ? "Boutique Air"
    

Periksa skrip HTML, data ada di `table` dengan `id="arrTable"`.

In [11]:
table = soup.find('table', {'id': 'arrTable'})
print(table)

<table id="arrTable">
<tr class="headerRow">
<th class="col1">Arriving From</th>
<th class="col2">Time</th>
<th class="col3">Airline</th>
<th class="col4">Flight</th>
<th class="col5">Claim</th>
<th class="col6">Remarks</th>
</tr>
<tr class="rowOdd">
<td class="col1" id="SFB" onclick="setCityFilter('SFB', 'Orlando Sanford'); applyFilters()">Orlando Sanford</td>
<td class="col2"> 7:55 AM</td>
<td class="col3" id="G4" onclick="setAirlineFilter('G4'); applyFilters()">
<object data="https://aerocloud-us-fids-widgets.s3.us-east-2.amazonaws.com/G4_whiteundefined.png" type="image/png">
<img src="https://aerocloud-us-fids-widgets.s3.us-east-2.amazonaws.com/G4_white.png" title=""/></object></td>
<td class="col4">1195</td>
<td class="claim col5"></td>
<td class="col6"><div class="cancelled">Cancelled</div></td>
</tr><tr class="rowEven">
<td class="col1" id="CLT" onclick="setCityFilter('CLT', 'Charlotte'); applyFilters()">Charlotte</td>
<td class="col2"> 10:45 AM</td>
<td class="col3" id="AA" onc

Dapatkan semua baris data, pada web ini menggunakan tag `tr`

In [12]:
trows = table.find_all('tr')
trows[0]

<tr class="headerRow">
<th class="col1">Arriving From</th>
<th class="col2">Time</th>
<th class="col3">Airline</th>
<th class="col4">Flight</th>
<th class="col5">Claim</th>
<th class="col6">Remarks</th>
</tr>

Data berada di tag `td`. Tinggal lakukan loop untuk mendapatkan semua data.

In [14]:
print('{:20} | {:15s} | {:15s} | {:15s} | {:10s}'.format(
    'Arriving From', 'Time', 'Flight', 'Claim', 'Flight'))
for i, trow in enumerate(trows):
    titems = trow.find_all('td')   # find all the data items in each row
    # Check if enough columns exist before accessing them
    if len(titems) >= 6:  # Adjusted to 6 to access titems[5] safely
        try:
            print('{:20s} | {:15s} | {:15s} | {:15s} | {:10s}'.format(
                titems[0].contents[0],          # contents of the 0 table item (column)
                titems[1].contents[0],          # contents of the 1 table item
                titems[3].contents[0],          # contents of the 3 table item
                titems[4].contents[0],          # contents of the 4 table item
                str(titems[5].contents[0].contents[0]) if titems[5].contents else ''  # Check if titems[5] has contents before accessing
            ))
        except IndexError:
            print(f"Skipping row {i} due to missing data in a column")
            # Or handle the error in another appropriate way
    else:
        print(f"Skipping row {i} due to insufficient columns")

Arriving From        | Time            | Flight          | Claim           | Flight    
Skipping row 0 due to insufficient columns
Skipping row 1 due to missing data in a column
Charlotte            |  10:45 AM       | 5520            | 2               | On Time   
Chicago O'Hare       |  10:46 AM       | 5500            | 2               | On Time   
Chicago O'Hare       |  10:54 AM       | 3585            | 2               | On Time   
Minneapolis - St Paul |  11:11 AM       | 3602            | 1               | On Time   
Ronald Reagan Washington Nat'l |  11:59 AM       | 5472            | 2               | On Time   
Dallas - Fort Worth  |  12:27 PM       | 1905            | 2               | On Time   
Denver International |  1:50 PM        | 4386            | 1               | On Time   
Phoenix Sky Harbor   |  2:06 PM        | 4895            | 2               | On Time   
Denver International |  2:09 PM        | 1241            | 2               | On Time   
Chicago O'Hare     

Sekarang, kita sudah mendapatkan data penerbangan.

## 3. Get Live Stock Price using `yahoo_fin` API

Seperti yang telah kita lihat sebelumnya, menulis kode scraping/crawling dari awal melibatkan banyak pekerjaan kasar dan tidak rapi. Namun, dalam banyak kasus, ada orang-orang yang telah melalui semua itu dan dengan murah hati memutuskan untuk membuat serangkaian fungsi praktis yang memungkinkan Anda melewati semua kerepotan tersebut. Atau, terkadang, para insinyur dan pengembang di perusahaan, yang sebenarnya membangun halaman web dan tahu persis bagaimana informasi tersebut diatur, memutuskan untuk memberikan cara kepada "pengguna nerd" seperti saya untuk mengakses data mereka. Apapun alasannya, API kumpulan data pada dasarnya adalah serangkaian fungsi yang telah ditentukan sebelumnya yang membantu Anda mengakses data.

Dalam contoh cepat ini, kita akan mengambil data harga saham secara real-time menggunakan API Yahoo! Finance (`yahoo_fin`). Pertama-tama, mari kita instal API `yahoo_fin`.

In [15]:
!pip install --upgrade yahoo_fin

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=8488565c172657ebe250736ec9553ebaf05890ef4ac0548327e4b1700daf3f52
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


`yahoo_fin` hadir dengan banyak modul di dalamnya. Di antaranya, dalam contoh ini, kita akan menggunakan modul stock_info. Cara mengimpornya seharusnya terlihat seperti ini:

In [1]:
from yahoo_fin import stock_info as si

Untuk API kumpulan data, kita tidak akan terlalu mendalami detailnya, tetapi berikut beberapa hal yang bisa Anda lakukan untuk mengambil informasi saham secara real-time.

In [2]:
# Get Netflix (NFLX) stock info from year 2015 to 2018
data = si.get_data('NFLX', start_date='01/01/2015', end_date='12/31/2018')
data

,open,high,low,close,adjclose,volume,ticker
2015-01-02,49.151428,50.331429,48.731430,49.848572,49.848572,13475000,NFLX
2015-01-05,49.258572,49.258572,47.147144,47.311428,47.311428,18165000,NFLX
2015-01-06,47.347141,47.639999,45.661430,46.501431,46.501431,16037700,NFLX
2015-01-07,47.347141,47.421429,46.271427,46.742859,46.742859,9849700,NFLX
2015-01-08,47.119999,47.835712,46.478573,47.779999,47.779999,9601900,NFLX
...,...,...,...,...,...,...,...
2018-12-21,263.829987,264.500000,241.289993,246.389999,246.389999,21397600,NFLX
2018-12-24,242.000000,250.649994,233.679993,233.880005,233.880005,9547600,NFLX
2018-12-26,233.919998,254.500000,231.229996,253.669998,253.669998,14402700,NFLX
2018-12-27,250.110001,255.589996,240.100006,255.570007,255.570007,12235200,NFLX


In [3]:
data.index # gives time stamps

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2018-12-14', '2018-12-17', '2018-12-18', '2018-12-19',
               '2018-12-20', '2018-12-21', '2018-12-24', '2018-12-26',
               '2018-12-27', '2018-12-28'],
              dtype='datetime64[ns]', length=1005, freq=None)

In [4]:
data['volume'].values  # gives values of the column named 'volume'

array([13475000, 18165000, 16037700, ..., 14402700, 12235200, 10992800])

In [5]:
data[['open','close']].values # gives multiple columns

array([[ 49.15142822,  49.84857178],
       [ 49.25857162,  47.31142807],
       [ 47.34714127,  46.50143051],
       ...,
       [233.91999817, 253.66999817],
       [250.11000061, 255.57000732],
       [257.94000244, 256.07998657]])

## 4. Conclusion

Kita telah melihat beberapa contoh dalam notebook ini tentang cara mengumpulkan data dari (1) repositori data publik, (2) web scraping manual, dan (3) fungsi API kumpulan data. Sebenarnya masih banyak hal lain yang perlu dibahas hanya untuk topik data mining ini, dan apa yang baru saja Anda lihat di atas hanyalah sebagian kecil. Tapi jangan khawatir. Selama Anda memahami prinsip-prinsip di atas, mempelajari alat/metode lain tidak akan menjadi tantangan besar, hanya perlu sedikit investasi waktu untuk mengenal semua perintah dan sebagainya.

# TUGAS
- Download file ipynb
- Ubah nama file jadi Tugas3_NIM_Nama.ipynb
- Upload di: https://docs.google.com/forms/d/e/1FAIpQLScabF6OTtWCFljQXTNOsw4vxj19vPa0SDQsQ5RDUSbk5oGM4w/viewform?usp=share_link

### 1. Mencari data di Kaggle
- Cari data tentang harga di kaggle, misalkan harga komputer, harga emas, harga berlian, dll. Gunakan kata kunci misalkan `cold price`
- Download data tersebut
- Load dengan Spark Dataframe
- Tampilkan rangkuman data statistik, seperti:
    - Jumlah data
    - Harga tertinggi
    - Harga terendah
    - dll

In [6]:
!kaggle datasets download -d budnyak/wine-rating-and-price -p data

Dataset URL: https://www.kaggle.com/datasets/budnyak/wine-rating-and-price
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
  0% 0.00/345k [00:00<?, ?B/s]
100% 345k/345k [00:00<00:00, 97.0MB/s]


In [8]:
import platform
import os

if platform.system() == 'Windows':
    this_file_path = !echo %cd%
    data_path = os.path.join(*[this_file_path[0], 'data'])

In [10]:
import zipfile

zip_path = '/content/data/wine-rating-and-price.zip'

extract_path = '/content/data/ekstrak/'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [11]:
import pandas as pd

csv_files = [f for f in os.listdir(extract_path) if f.endswith('.csv')]
print("File CSV yang ditemukan:", csv_files)

File CSV yang ditemukan: ['Rose.csv', 'Red.csv', 'Sparkling.csv', 'White.csv', 'Varieties.csv']


In [12]:
df = os.path.join(extract_path, 'Red.csv')

df = pd.read_csv(df)
df.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8666 entries, 0 to 8665
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8666 non-null   object 
 1   Country          8666 non-null   object 
 2   Region           8666 non-null   object 
 3   Winery           8666 non-null   object 
 4   Rating           8666 non-null   float64
 5   NumberOfRatings  8666 non-null   int64  
 6   Price            8666 non-null   float64
 7   Year             8666 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 541.8+ KB


In [13]:
df.describe()

,Rating,NumberOfRatings,Price
count,8666.000000,8666.000000,8666.000000
mean,3.890342,415.287445,39.145065
std,0.308548,899.726373,84.936307
min,2.500000,25.000000,3.550000
25%,3.700000,66.000000,10.680000
50%,3.900000,157.000000,18.200000
75%,4.100000,401.000000,38.142500
max,4.800000,20293.000000,3410.790000


In [14]:
max_price = df['Price'].max()
min_price = df['Price'].min()

max_rating = df['Rating'].max()
min_rating = df['Rating'].min()

print("\nHarga tertinggi:", max_price)
print("Harga terendah:", min_price)
print("Rating tertinggi:", max_rating)
print("Rating terendah:", min_rating)


Harga tertinggi: 3410.79
Harga terendah: 3.55
Rating tertinggi: 4.8
Rating terendah: 2.5


In [16]:
ratings_per_year = df.groupby('Year')['NumberOfRatings'].sum().reset_index()

ratings_per_year.columns = ['Year', 'Total Ratings']
print("Jumlah peminat anggur setiap tahunnya:")
ratings_per_year.head(10)

Jumlah peminat anggur setiap tahunnya:


,Year,Total Ratings
0,1988,203
1,1989,2150
2,1990,726
3,1991,603
4,1992,1218
5,1993,1717
6,1995,624
7,1996,353
8,1997,2122
9,1998,3966


In [17]:
ratings_per_year_wine = df.groupby(['Year', 'Name', 'Country', 'Price'])['NumberOfRatings'].sum().reset_index()

most_popular_wine_per_year = ratings_per_year_wine.loc[ratings_per_year_wine.groupby('Year')['NumberOfRatings'].idxmax()]

most_popular_wine_per_year.columns = ['Year', 'Wine Name', 'Country', 'Price', 'Total Ratings']

print("Anggur yang paling diminati setiap tahunnya:")
most_popular_wine_per_year.head(10)

Anggur yang paling diminati setiap tahunnya:


,Year,Wine Name,Country,Price,Total Ratings
0,1988,Hermitage La Chapelle Rouge 1988,France,209.60,203
1,1989,Pauillac (Premier Grand Cru Classé) 1989,France,1168.77,2069
4,1990,Saint-Julien (Grand Cru Classé) 1990,France,252.47,589
5,1991,Pauillac (Premier Grand Cru Classé) 1991,France,866.59,603
7,1992,Pauillac (Premier Grand Cru Classé) 1992,France,1071.29,840
9,1993,Pauillac (Premier Grand Cru Classé) 1993,France,1051.80,1717
11,1995,Hermitage La Chapelle Rouge 1995,France,181.75,378
14,1996,Crozes-Hermitage Domaine de Thalabert 1996,France,54.25,122
25,1997,Tignanello 1997,Italy,574.15,957
27,1998,Rouge (Gaston Hochar) 1998,Lebanon,37.92,2208


### 2. Web crawler
- Ekstraksi informasi dosen Informatika UMM dari halaman berikut: https://sinta.kemdikbud.go.id/departments/authors/2053/6399D77D-8C55-4DFA-80F8-70027523099E/80BC8D5A-DA8E-434B-87D9-687BF81DBDA4
- Informasi yang diekstraksi adalah:
  - Nama dosen
  - SINTA Score 3Yr
  - SINTA Score
  - Affil Score 3Yr
  - Affil Score
- Tampilkan dalam bentuk printout berformat tabel seperti contoh di tutorial
- Perhatikan bahwa pada laman web tersebut, tidak digunakan tag `table`, `tr`, dan `td`. Semua data berada di tag `div` namun dengan nilai atribut `class` yang berbeda-beda. Tag dan atribut inilah yang harus digunakan untuk pencarian.

In [24]:
url = 'https://sinta.kemdikbud.go.id/departments/authors/2053/6399D77D-8C55-4DFA-80F8-70027523099E/80BC8D5A-DA8E-434B-87D9-687BF81DBDA4'

In [29]:
import requests

response = requests.get(url, timeout=10)
response.raise_for_status()

In [30]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, 'html.parser')

In [31]:
data_list = []

nama_divs = soup.find_all('div', class_='profile-name')
for nama_div in nama_divs:
    # Nnama dosen
    nama_dosen = nama_div.find('a').get_text(strip=True) if nama_div.find('a') else ''  # Karena nama dosen berada pada tag <a>

    # Memuat skor SINTA dan Affil
    scores = nama_div.find_next('div', class_='row').find_all('div', class_='col')

    # Mengambil data dari div sesuai urutan
    sinta_score_3yr = scores[0].find('div', class_='stat-num text-center').get_text(strip=True)
    sinta_score = scores[1].find('div', class_='stat-num text-center').get_text(strip=True)
    affil_score_3yr = scores[2].find('div', class_='stat-num text-center').get_text(strip=True)
    affil_score = scores[3].find('div', class_='stat-num text-center').get_text(strip=True)

    # Menambahkan data ke dalam list
    data_list.append({
        'Nama Dosen': nama_dosen,
        'SINTA Score 3Yr': sinta_score_3yr,
        'SINTA Score': sinta_score,
        'Affil Score 3Yr': affil_score_3yr,
        'Affil Score': affil_score
    })

df = pd.DataFrame(data_list)

print(df.to_markdown(index=False))

| Nama Dosen                |   SINTA Score 3Yr |   SINTA Score |   Affil Score 3Yr |   Affil Score |
|:--------------------------|------------------:|--------------:|------------------:|--------------:|
| AGUS EKO MINARNO          |               594 |         1.41  |                 0 |             0 |
| YUFIS AZHAR               |               544 |         1.314 |                 0 |             0 |
| WAHYU ANDHYKA KUSUMA      |               381 |         1.091 |                 0 |             0 |
| GALIH WASIS WICAKSONO     |               363 |       691     |                 0 |             0 |
| YUDA MUNARKO              |               312 |       696     |                 0 |             0 |
| GITA INDAH MARTHASARI     |               308 |       683     |                 0 |             0 |
| NUR HAYATIN               |               274 |       774     |                 0 |             0 |
| ZAMAH SARI                |               273 |       793     |                 